In [1]:
try:
    from urllib.parse import urlparse, urlencode
    from urllib.request import urlopen, Request
    from urllib.error import HTTPError
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode
    from urllib2 import urlopen, Request, HTTPError

In [2]:
gbxml_address = 'https://rawgit.com/ladybug-tools/spider/master/gbxml-sample-files/bristol-clifton-downs-fixed.xml'

In [3]:
request = urlopen(gbxml_address)

gbxml_string = request.read()

with open('bristol-clifton-downs-fixed.xml', 'w') as f:
    f.write(gbxml_string)

In [4]:
from xml.dom import minidom

gbxml = minidom.parseString(gbxml_string)
gbxml_spaces = gbxml.getElementsByTagName('Space')
gbxml_surfaces = gbxml.getElementsByTagName('Surface')


## Parse Surfaces into HBSurfaces

In [5]:
from lib.gbxml_surface_parser import get_hb_surface

hb_surfaces = []

# Check space 'conditionType' to be 'HeatedAndCooled', if 'unconditioned' then do not use.

for surface in gbxml_surfaces:
    hb_surface = get_hb_surface(surface)
    if hb_surface is not None:
        hb_surfaces.append(hb_surface)
    
len(hb_surfaces)

Path to OpenStudio is set to: C:/Program Files/OpenStudio 1.14.0
Path to radiance is set to: c:/radiance
Path to perl is set to: C:/Program Files/OpenStudio 1.14.0\strawberry-perl-5.16.2.1-32bit-portable-reduced


517

In [7]:
# fenestrated = []

# for surface in hb_surfaces:
# #     print(len(surface.children_surfaces))
#     if hb_surface.has_child_surfaces:
#         print(len(surface.children_surfaces))

# #         fenestrated.append(hb_surface)
        
# # fenestrated[0].children_surfaces

# fenestrated

In [9]:
hb_surfaces[0].to_json()

{'name': 'Basic_Wall__SIM_INT_SLD_Core__883750_',
 'surface_material': {'b_reflectance': 0.5,
  'g_reflectance': 0.5,
  'modifier': 'void',
  'name': 'generic_wall',
  'r_reflectance': 0.5,
  'roughness': 0.0,
  'specularity': 0.0,
  'type': 'plastic'},
 'surface_type': 0.0,
 'vertices': [[(497.6739, 1030.624, 19.25),
   (497.7677, 1032.187, 19.25),
   (497.7677, 1032.187, 23.15),
   (497.6739, 1030.624, 23.15)]]}

## Parse Zones into Analysis Grids

In [10]:
from lib.gbxml_surface_parser import get_points, get_clean_name
from lib.gbxml_room_grid_parser import grid_from_floor, points_in_floor_polygon
from honeybee.radiance.analysisgrid import AnalysisGrid

analysis_grids = []

for space in gbxml_spaces:
    space_name_raw = space.getElementsByTagName('Name')[0].childNodes[0].nodeValue
    name = get_clean_name(space_name_raw)
    floor_points = get_points(space)
    grid = grid_from_floor(floor_points)
    
    analysis_grid = AnalysisGrid.from_points_and_vectors(grid, name=name)

    analysis_grids.append(analysis_grid)

len(analysis_grids)

87

### IEQc: 8.1 Daylight and Views – Daylight
Demonstrate through computer simulation that the applicable spaces achieve daylight
illuminance levels of a minimum of 10 footcandles (fc) (110 Lux) and a maximum of 500 fc
(5,400 Lux) in a clear sky condition on September 21 at 9 a.m. and 3 p.m.; areas
Provide glare control devices to avoid high-contrast situations that could impede visual
tasks. However, designs that incorporate view-preserving automated shades for glare
control may demonstrate compliance for only the minimum 10 fc (110 lux) illuminance level. 

In [11]:
from honeybee.radiance.recipe.pointintime.gridbased import GridBased as PiTGridBased
from honeybee.radiance.sky.climatebased import ClimateBased as ClimateBasedSky
from ladybug.wea import Wea

epw_file_path = 'data/GBR_Birmingham.035340_IWEC.epw'

wea = Wea.from_epw_file(epw_file_path)

sky_1 = ClimateBasedSky.from_wea(wea=wea, month=9, day=21, hour=9, north=0)

sky_2 = ClimateBasedSky.from_wea(wea=wea, month=9, day=21, hour=15, north=0)

In [18]:
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as DFGridBased
import os

df_recipe = DFGridBased(analysis_grids=analysis_grids, hb_objects=hb_surfaces)


target_folder = os.path.join(os.getcwd(), 'recipes')

command_path = df_recipe.write(target_folder, 'test')

df_recipe.run(command_path)

Found 517 opaque surfaces.
Found 268 fenestration surfaces.
Found 0 window-groups.
Writing recipe contents to: C:\Users\Antoine\Documents\projects\honeybee tutorials\recipes\test\daylightfactor


True

In [22]:
analysis_grids = df_recipe.results()

analysis_grids[0].to_json()

Unloading the current values from the analysis grids.


{'analysis_points': [{'direction': (0.0, 0.0, 1.0),
   'location': (497.2521, 1031.151, 20.0),
   'values': [[defaultdict(<function honeybee.radiance.analysispoint.double>,
                 {379440: [0.0, None]})]]},
  {'direction': (0.0, 0.0, 1.0),
   'location': (497.2521, 1031.651, 20.0),
   'values': [[defaultdict(<function honeybee.radiance.analysispoint.double>,
                 {379440: [0.0, None]})]]},
  {'direction': (0.0, 0.0, 1.0),
   'location': (497.2521, 1032.151, 20.0),
   'values': [[defaultdict(<function honeybee.radiance.analysispoint.double>,
                 {379440: [0.0, None]})]]}],
 'name': u'00_100_Riser_2'}

In [14]:
analysis_grid = analysis_grids[10]

recipe_1 = PiTGridBased(sky=sky_1, analysis_grids=[analysis_grid], hb_objects=hb_surfaces)

recipe_2 = PiTGridBased(sky=sky_2, analysis_grids=[analysis_grid], hb_objects=hb_surfaces)


Found 517 opaque surfaces.
Found 268 fenestration surfaces.
Found 0 window-groups.
Found 517 opaque surfaces.
Found 268 fenestration surfaces.
Found 0 window-groups.


In [15]:
import os

target_folder = os.path.join(os.getcwd(), 'recipes')

command_path = recipe_1.write(target_folder, 'test')
recipe_1.run(command_path)

Writing recipe contents to: C:\Users\Antoine\Documents\projects\honeybee tutorials\recipes\test\gridbased
Radiance path C:\Program Files\OpenStudio 1.14.0\share\openStudio\Radiance\bin has a whitespace. Some of the radiance commands may fail.
We strongly suggest you to install radiance under a path with no withspace (e.g. c:/radiance)


True